In [ ]:
#!gdown --id 1yhbk1BJUAnL6_XXYL2KC5u7k7JzGMu3I

In [ ]:
#!unzip RainTrainH.zip

In [ ]:
import numpy as np

import torchvision.transforms as transforms
#from torchvision.utils import make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
# data (path)
#dataset_name = 'RainTrainH'
#root = './'+dataset_name

N_CPU = 1
# data (img)
IMG_HEIGHT = 512
IMG_WIDTH = 512
CHANNELS = 3

# training
EPOCH_START = 0 # epoch to start training from
EPOCH_NUM   = 50 # number of epochs of training
BATCH_SIZE = 1 # size of the batches
LR = 0.0002 # adam : learning rate
BETA1 = 0.5 # adam : decay of first order momentum of gradient
BETA2 = 0.999 # adam : decay of first order momentum of gradient
EPOCH_DECAY = 25 # suggested default : 100 (suggested 'n_epochs' is 200)
                 # epoch from which to start lr decay

# 1. read Data from folder

In [ ]:
print("======================read Data from folder===============================")

### 2-a load data

In [ ]:
import glob
import os
#######################################################################################
def load_train_data(img_path, n_len, valid_ratio=0.12):
    train_image = [i for i in sorted(glob.glob(os.path.join(img_path, "*.*")))]
    train_data = list(zip(train_image))
    #np.random.seed(1234)
    #np.random.shuffle(train_data)
    #split_len = int(len(train_data) * valid_ratio)
    #print("split_len: ",split_len)
    
    train_set = train_data
    valid_set = train_data[:n_len]
    
    return train_set, valid_set
#######################################################################################
def load_test_data(img_path):
    test_image = [i for i in sorted(glob.glob(os.path.join(img_path, "*.*")))]
    test_data = list(zip(test_image))
    test_set = test_data
    return test_set

In [ ]:
import glob
import os
TRA_PATHA = "./RainTrainH/train/rainy_image"
TRA_PATHB = "./RainTrainH/train/ground_truth"
TST_PATHA = "./RainTrainH/test/rainy_image"
TST_PATHB = "./RainTrainH/test/ground_truth"
CKP_PATHA = ""
trainA_set, validA_set = load_train_data(TRA_PATHA,n_len=300)
trainB_set, validB_set = load_train_data(TRA_PATHB,n_len=50)
testA_set = load_test_data(TST_PATHA)

#trainB_set, validB_set = load_train_data(TRA_PATHB)

In [ ]:
len(validA_set)

In [ ]:
len(validB_set)

### 2-b custom datasets

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

transforms_train = [
    #transforms.Resize(int(img_height*1.12), Image.BICUBIC),
    #transforms.RandomCrop((IMG_HEIGHT//16, IMG_WIDTH//16)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]
transforms_train = transforms.Compose(transforms_train)

transforms_test = [
    #transforms.Resize(int(img_height*1.12), Image.BICUBIC),
    #transforms.RandomCrop((img_height, img_width)),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]
transforms_test = transforms.Compose(transforms_test)


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#######################################################################################
class TraValDataset(Dataset):
    def __init__(self, dataA, dataB, augment=None, unaligned=False):
        self.dataA      = dataA
        self.dataB      = dataB
        self.augment   = augment
        self.unaligned = unaligned

    def __len__(self):
        return max(len(self.dataA), len(self.dataB))
    
    def normalize(img):
        # TODO: do normalization there
        pass
    
    def read_img(self, path):
        img = Image.open(path).convert('RGB')
        #img = img.resize((224,224))
        #img = transforms.Resize([256,256])(img)
        #img = transforms.Pad(32, padding_mode='symmetric')(img)
        if (max(IMG_WIDTH,IMG_HEIGHT)>=max(img.size[1],img.size[0])):
            pad_h    = (IMG_WIDTH-img.size[1])//2
            pad_h_r  = (IMG_WIDTH-img.size[1])%2
            pad_w    = (IMG_HEIGHT-img.size[0])//2
            pad_w_r  = (IMG_HEIGHT-img.size[0])%2
            img_pad  = transforms.Pad(( pad_w, pad_h, pad_w+pad_w_r, pad_h+pad_h_r), fill=255, padding_mode="constant")(img)
        else:
            img_pad  = img 
        if not self.augment is None:
            img_aug = self.augment(img_pad)
        else:
            img_aug = img_pad
         
        return img_aug
    
    def __getitem__(self, idx):
        img_realX = self.read_img(self.dataA[idx % len(self.dataA)][0])
        if self.unaligned:
            #image_B = Image.open(self.files_B[np.random.randint(0, len(self.files_B)-1)])
            img_realY = self.read_img(self.dataB[idx//6][0])
        else:
            img_realY = self.read_img(self.dataB[idx % len(self.dataB)][0])
        return {'A': img_realX, 'B': img_realY}
    
#######################################################################################
class TestDataset(Dataset):
    def __init__(self, dataA, augment=None, unaligned=False):
        self.dataA      = dataA
        self.augment   = augment

    def __len__(self):
        return len(self.dataA)
    
    def normalize(img):
        # TODO: do normalization there
        pass
    
    def read_img(self, path):
        img = Image.open(path).convert('RGB')
        #img = img.resize((224,224))
        #img = transforms.Resize([256,256])(img)
        #img = transforms.Pad(32, padding_mode='symmetric')(img)
        if (max(IMG_WIDTH,IMG_HEIGHT)>=max(img.size[1],img.size[0])):
            pad_h    = (IMG_WIDTH-img.size[1])//2
            pad_h_r  = (IMG_WIDTH-img.size[1])%2
            pad_w    = (IMG_HEIGHT-img.size[0])//2
            pad_w_r  = (IMG_HEIGHT-img.size[0])%2
            img_pad  = transforms.Pad(( pad_w, pad_h, pad_w+pad_w_r, pad_h+pad_h_r), fill=255, padding_mode="constant")(img)
        else:
            img_pad  = img 
        if not self.augment is None:
            img_aug = self.augment(img_pad)
        else:
            img_aug = img_pad
        return img_aug
    
    def __getitem__(self, idx):
        img_realX = self.read_img(self.dataA[idx % len(self.dataA)][0])
        img_realXName = self.dataA[idx][0].split("/")[-1]
        return {'A': img_realX, 'Name': img_realXName}

In [ ]:
(512-481)%2

In [ ]:
train_dataset = TraValDataset(trainA_set, trainB_set, augment=transforms_train, unaligned=True)#, transform)
#trainB_dataset = TraValDataset(trainB_set)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_CPU)
#trainB_loader = DataLoader(trainB_dataset, batch_size=16, shuffle=True)

valid_dataset = TraValDataset(validA_set, validB_set, augment=transforms_test, unaligned=True)
#validB_dataset = TraValDataset(validB_set)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_CPU)
#valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

test_dataset = TestDataset(testA_set, augment=transforms_test, unaligned=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_CPU)

# 2. Module

### 2-a Generator

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # Pads the input tensor using the reflection of the input boundary
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features), 
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)


class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_block):
        super(GeneratorResNet, self).__init__()
        
        channels = input_shape[0]
        
        # Initial Convolution Block
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True)
        ]
        in_features = out_features
        
        # Downsampling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
        
        # Residual blocks
        for _ in range(num_residual_block):
            model += [ResidualBlock(out_features)]
            
        # Upsampling
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2), # --> width*2, heigh*2
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            
        # Output Layer
        model += [nn.ReflectionPad2d(channels),
                  nn.Conv2d(out_features, channels, 7),
                  nn.Tanh()
                 ]
        
        # Unpacking
        self.model = nn.Sequential(*model) 
        
    def forward(self, x):
        return self.model(x)

### 2-b Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        
        channels, height, width = input_shape
        
        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height//2**4, width//2**4)
        
        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128,256),
            *discriminator_block(256,512),
            nn.ZeroPad2d((1,0,1,0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )
        
    def forward(self, img):
        return self.model(img)

In [ ]:
input_shape = (CHANNELS, IMG_HEIGHT, IMG_WIDTH) # (3,256,256)
n_residual_blocks = 9 # suggested default, number of residual blocks in generator

# 3. Loss Function

In [ ]:
GAN_LOSS      = torch.nn.MSELoss()
CYCLE_LOSS    = torch.nn.L1Loss()
IDENTITY_LOSS = torch.nn.L1Loss()

# 4. Train Process

In [ ]:
print("========================= Train process ===============================")

### 4-a weight initial

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02) # reset Conv2d's weight(tensor) with Gaussian Distribution
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0) # reset Conv2d's bias(tensor) with Constant(0)
        elif classname.find('BatchNorm2d') != -1:
            torch.nn.init.normal_(m.weight.data, 1.0, 0.02) # reset BatchNorm2d's weight(tensor) with Gaussian Distribution
            torch.nn.init.constant_(m.bias.data, 0.0) # reset BatchNorm2d's bias(tensor) with Constant(0)

### 4-b Train

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
DEVICE_ID = 0
torch.cuda.set_device(DEVICE_ID)
USE_GPU = torch.cuda.is_available()
device = torch.device("cuda" if USE_GPU else "cpu")
import torch
from torchmetrics.image import StructuralSimilarityIndexMeasure
#from piqa import PSNR, SSIM

import skimage
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR
################################################################################
def train(train_loader, G_AB, G_BA, D_A, D_B, 
          IDENTITY_LOSS, GAN_LOSS, CYCLE_LOSS,
          use_gpu=True):
    train_loss = []
    #train_acc = []
    train_psnr = []
    train_ssim = []
    for idx, batch in enumerate(tqdm(train_loader)):
        if use_gpu:
            img_realX = batch['A'].to(device)
            img_realY = batch['B'].to(device)
            valid = torch.cuda.FloatTensor(np.ones((img_realX.size(0), *D_A.output_shape))) # requires_grad = False. Default.
            fake = torch.cuda.FloatTensor(np.zeros((img_realX.size(0), *D_A.output_shape))) # requires_grad = False. Default.
        G_AB.train()
        G_BA.train()
        ####################################
        # Generator
        optimizer_G.zero_grad()

        img_fakeY = G_AB(img_realX)
        img_cycledX = G_BA(img_fakeY)
        img_fakeX = G_BA(img_realY)
        img_cycledY = G_AB(img_fakeX)

        disc_realX = D_A(img_realX)
        disc_realY = D_B(img_realY)
        disc_fakeX = D_A(img_fakeX)
        disc_fakeY = D_B(img_fakeY)

        # Identity Loss
        id_loss_X = IDENTITY_LOSS(img_fakeX, img_realX)
        id_loss_Y = IDENTITY_LOSS(img_fakeY, img_realY)
        id_loss_t = (id_loss_X+id_loss_Y)/2
        # GAN Loss
        gan_loss_XY = GAN_LOSS(disc_fakeY, valid)
        gan_loss_YX = GAN_LOSS(disc_fakeX, valid)
        gan_loss_t  = (gan_loss_XY+gan_loss_YX)/2
        # Cycle Loss
        cyc_loss_X = CYCLE_LOSS(img_cycledX, img_realX)
        cyc_loss_Y = CYCLE_LOSS(img_cycledY, img_realX)
        cyc_loss_t = (cyc_loss_X+cyc_loss_Y)/2
        # Generator Total Loss
        loss_G = (5.0*id_loss_t) + gan_loss_t + (10.0*cyc_loss_t)

        loss_G.backward()
        optimizer_G.step()
        ####################################
        # Discriminator A
        optimizer_D_A.zero_grad()

        real_loss_X = GAN_LOSS(disc_realX, valid)
        fake_loss_X = GAN_LOSS(disc_fakeX.detach(), fake)
        loss_D_A = (real_loss_X+fake_loss_X)/2
        
        loss_D_A.backward() 
        optimizer_D_A.step()
        ####################################
        # Discriminator B
        optimizer_D_B.zero_grad()

        real_loss_Y = GAN_LOSS(disc_realY, valid)
        fake_loss_Y = GAN_LOSS(disc_fakeY.detach(), fake)
        loss_D_B = (real_loss_Y+fake_loss_Y)/2

        loss_D_B.backward() 
        optimizer_D_B.step()
        ####################################
        loss_D = (loss_D_A+loss_D_B)/2
        loss = loss_G + loss_D 
        #loss.backward(retain_graph=True)
        ####################################
        #with torch.no_grad():
        #print(disc_fakeY)
        psnr = PSNR(img_fakeY.detach()[0].cpu().numpy(),img_realY.detach()[0].cpu().numpy())
        #print(psnr.item())
        ssim = SSIM(img_fakeY.detach()[0].permute(1,2,0).cpu().numpy(),img_realY.detach()[0].permute(1,2,0).cpu().numpy(),multichannel=True) 
        #ignite.metrics.SSIM
        #predict = torch.argmax(disc_fakeY, dim=-1)
        #original = torch.argmax(disc_realY, dim=-1)
        #acc = np.mean((original == predict).cpu().numpy())
        #train_acc.append(acc)
        train_psnr.append(psnr)
        train_ssim.append(ssim)
        train_loss.append(loss.item())
        print('[Train Epoch %d/%d] [Batch %d/%d] [D loss : %4f] [G loss : %4f - (adv : %4f, cycle : %4f, identity : %4f)]'
                %(epoch+1,EPOCH_NUM,       # [Epoch -]
                    idx+1,len(train_loader),   # [Batch -]
                    loss_D.item(),       # [D loss -]
                    loss_G.item(),       # [G loss -]
                    gan_loss_t.item(),     # [adv -]
                    cyc_loss_t.item(),   # [cycle -]
                    cyc_loss_t.item(),# [identity -]
                    ))
################################################################################
def valid(valid_loader, G_AB, G_BA, D_A, D_B, 
          IDENTITY_LOSS, GAN_LOSS, CYCLE_LOSS,
          use_gpu=True):
    #with torch.no_grad():
    valid_loss = []
    #valid_acc = []
    valid_psnr = []
    valid_ssim = []
    for idx, batch in enumerate(tqdm(valid_loader)):
        if use_gpu:
            img_realX = batch['A'].to(device)
            img_realY = batch['B'].to(device)
            valid = torch.cuda.FloatTensor(np.ones((img_realX.size(0), *D_A.output_shape))) # requires_grad = False. Default.
            fake = torch.cuda.FloatTensor(np.zeros((img_realX.size(0), *D_A.output_shape))) # requires_grad = False. Default.
        G_AB.eval()
        G_BA.eval()
        ####################################
        # Generator
        optimizer_G.zero_grad()

        img_fakeY = G_AB(img_realX)
        img_cycledX = G_BA(img_fakeY)
        img_fakeX = G_BA(img_realY)
        img_cycledY = G_AB(img_fakeX)

        disc_realX = D_A(img_realX)
        disc_realY = D_B(img_realY)
        disc_fakeX = D_A(img_fakeX)
        disc_fakeY = D_B(img_fakeY)

        # Identity Loss
        id_loss_X = IDENTITY_LOSS(img_fakeX, img_realX)
        id_loss_Y = IDENTITY_LOSS(img_fakeY, img_realY)
        id_loss_t = (id_loss_X+id_loss_Y)/2
        # GAN Loss
        gan_loss_XY = GAN_LOSS(disc_fakeY, valid)
        gan_loss_YX = GAN_LOSS(disc_fakeX, valid)
        gan_loss_t  = (gan_loss_XY+gan_loss_YX)/2
        # Cycle Loss
        cyc_loss_X = CYCLE_LOSS(img_cycledX, img_realX)
        cyc_loss_Y = CYCLE_LOSS(img_cycledY, img_realX)
        cyc_loss_t = (cyc_loss_X+cyc_loss_Y)/2
        # Generator Total Loss
        loss_G = (5.0*id_loss_t) + gan_loss_t + (10.0*cyc_loss_t)

        loss_G.backward()
        optimizer_G.step()
        ####################################
        # Discriminator A
        optimizer_D_A.zero_grad()

        real_loss_X = GAN_LOSS(disc_realX, valid)
        fake_loss_X = GAN_LOSS(disc_fakeX.detach(), fake)
        loss_D_A = (real_loss_X+fake_loss_X)/2

        loss_D_A.backward() 
        optimizer_D_A.step()
        ####################################
        # Discriminator B
        optimizer_D_B.zero_grad()

        real_loss_Y = GAN_LOSS(disc_realY, valid)
        fake_loss_Y = GAN_LOSS(disc_fakeY.detach(), fake)
        loss_D_B = (real_loss_Y+fake_loss_Y)/2
        
        loss_D_B.backward() 
        optimizer_D_B.step()
        ####################################
        loss_D = (loss_D_A+loss_D_B)/2
        loss = loss_G + loss_D 
        #loss.backward(retain_graph=True)
        ####################################
        #print(disc_fakeY)
        psnr = PSNR(img_fakeY.detach()[0].cpu().numpy(),img_realY.detach()[0].cpu().numpy())
        #print(psnr.item())
        ssim = SSIM(img_fakeY.detach()[0].permute(1,2,0).cpu().numpy(),img_realY.detach()[0].permute(1,2,0).cpu().numpy(),multichannel=True) 
        #ignite.metrics.SSIM
        #predict = torch.argmax(disc_fakeY, dim=-1)
        #original = torch.argmax(disc_realY, dim=-1)
        #acc = np.mean((original == predict).cpu().numpy())
        #train_acc.append(acc)
        valid_psnr.append(psnr)
        valid_ssim.append(ssim)
        valid_loss.append(loss.item())
        print('[Valid Epoch %d/%d] [Batch %d/%d] [D loss : %4f] [G loss : %4f - (adv : %4f, cycle : %4f, identity : %4f)]'
                %(epoch+1,EPOCH_NUM,       # [Epoch -]
                idx+1,len(valid_loader),   # [Batch -]
                loss_D.item(),       # [D loss -]
                loss_G.item(),       # [G loss -]
                gan_loss_t.item(),     # [adv -]
                cyc_loss_t.item(),   # [cycle -]
                cyc_loss_t.item(),# [identity -]
                ))
    return np.mean(valid_psnr), np.mean(valid_ssim), np.mean(valid_loss)

def save_checkpoint(valid_loss, loss_record, epoch, CKP_PATHA=CKP_PATHA, prefix='model'):
    # you can define the condition to save model :)
    #print(ssim_record[-5:].shape)
    #print(np.mean(ssim_record[-5:]))
    if valid_loss <= np.mean(loss_record[-5:]):    
        checkpoint_pathG_AB = CKP_PATHA+f'{"G_AB"}.pth'
        checkpoint_pathG_BA = CKP_PATHA+f'{"G_BA"}.pth'
        torch.save(G_AB.state_dict(), checkpoint_pathG_AB)
        print('model saved to %s' % checkpoint_pathG_AB)
        torch.save(G_BA.state_dict(), checkpoint_pathG_BA)
        print('model saved to %s' % checkpoint_pathG_BA)

def better(loss_record):
    if min(loss_record) == loss_record[-1]: return True
    return False

### 4-c valid

### 4-d Learing Schedule

In [ ]:
class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch
        
    def step(self, epoch):
        return 1.0 - max(0, epoch+self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

In [ ]:
# n_epochs = 10
# epoch = 0
# decay_epoch = 5

"""
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G,
    lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step
)

lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A,
    lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B,
    lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step
)
"""

### main

In [ ]:
import itertools
if __name__ == '__main__':
  G_AB = GeneratorResNet(input_shape, n_residual_blocks)
  G_BA = GeneratorResNet(input_shape, n_residual_blocks)
  #G = GeneratorResNet(input_shape, n_residual_blocks)
  D_A = Discriminator(input_shape)
  D_B = Discriminator(input_shape)
  G_AB.apply(weights_init)
  G_BA.apply(weights_init)
  D_A.apply(weights_init)
  D_B.apply(weights_init)
  #metric = StructuralSimilarityIndexMeasure()
  if USE_GPU:
    G_AB.to(device)
    G_BA.to(device)
    #G.to(device)
    D_A.to(device)
    D_B.to(device)
    #PSNR = PSNR()
    #SSIM = SSIM().to(device)
    #metric.to(device)
    #optimizerG_AB = torch.optim.Adam(G_AB.parameters(), lr=0.001)
    #optimizerG_BA = torch.optim.Adam(G_BA.parameters(), lr=0.001)
    optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=LR, betas=(BETA1,BETA2))
    optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=LR, betas=(BETA1,BETA2))
    optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=LR, betas=(BETA1,BETA2))
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.009, momentum=0.8)#lr=0.009
    #loss_fn = nn.CrossEntropyLoss()
    
    #acc_record = []
    #ssim_record = []
    loss_record = []
    
    for epoch in range(EPOCH_START, EPOCH_NUM):
      train(train_loader, G_AB, G_BA, D_A, D_B, 
          IDENTITY_LOSS, GAN_LOSS, CYCLE_LOSS,
          USE_GPU)
      valid_psnr,valid_ssim, valid_loss =  \
      valid(valid_loader, G_AB, G_BA, D_A, D_B, 
          IDENTITY_LOSS, GAN_LOSS, CYCLE_LOSS,
          USE_GPU)
      #acc_record.append(valid_acc)
      loss_record.append(valid_loss)
      #print("ssim_record: ",ssim_record.shape)
      #print("ssim_record: ",valid_ssim.shape)
        
      if better(loss_record):
        save_checkpoint(valid_loss, loss_record, epoch, prefix='model')
        
      print('########################################################')

# 5. Test Process

In [ ]:
DEVICE_ID = 0
torch.cuda.set_device(DEVICE_ID)
USE_GPU = torch.cuda.is_available()
device = torch.device("cuda" if USE_GPU else "cpu")
import torch
from torchmetrics.image import StructuralSimilarityIndexMeasure
#from piqa import PSNR, SSIM
import skimage
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR

In [ ]:
from torchvision.utils import save_image
def test(test_loader, model,TST_PATHB=TST_PATHB, use_gpu=True):
    with torch.no_grad():
        #predict_result = []
        #predict_name = []
        for idx, batch in enumerate(tqdm(test_loader)):
        #for (img_realX, img_realXName) in test_loader:
            if use_gpu:
                img_realX = batch['A'].to(device)
                img_realXName = batch['Name'][0]
            img_fakeY = model(img_realX)
            img_output = img_fakeY.squeeze()
            save_image(img_output, os.path.join(TST_PATHB,img_realXName),normalize=True)
            #predict = torch.argmax(output, dim=-1).tolist()
            #predict_result += predict
            #predict_name += name
    """    
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'label'])
        for id, r in zip(predict_name, predict_result):
            writer.writerow([id, r])
    """

In [ ]:
#del G_AB
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_AB.load_state_dict(torch.load('G_AB.pth'))
G_AB = G_AB.cuda()
test(test_loader, G_AB, TST_PATHB, USE_GPU)